In [1]:
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import numpy as np

import plotly 
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

#### RSI + MFI + Stochastic

In [214]:
#Forex
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv('./Forex Data/EURUSD_eatradingacademy/EURUSD5.csv', sep = '\t', header=None, names = colnames)
#df = pd.read_csv('./Forex Data/USDJPY/USDJPY1.csv', sep = '\t', header=None, names = colnames)

In [216]:
#df = df.iloc[:10000]

In [220]:
'''df['MA_50'] = ta.SMA(df['Close'],timeperiod=50)
df['EMA'] = ta.EMA(df['Close'], timeperiod = 5)
df['BB_up'],df['BB_mid'],df['BB_low'] = ta.BBANDS(df['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
df['TEMA_S'] = ta.TEMA(df['Close'], timeperiod = 12)
df['TEMA_L'] = ta.TEMA(df['Close'], timeperiod = 36)
df['Tamanho_candle'] = abs(df['Close'] - df['Open'])
df['Tamanho_total'] = abs(df['High'] - df['Low'])'''

def tendencia(x):
    if x['Close'] > x['MA_200']:
        return 'Alta'
    else:
        return 'Baixa'

def mfi_rsi_stochastic_tendencia(x):
    if x['RSI'] < 30 and x['MFI'] < 20 and x['slowk'] < 20 and x['Tendencia'] == 'Alta':
        return 'call'
    elif x['RSI'] > 70 and x['MFI'] > 80 and x['slowk'] > 80 and x['Tendencia'] == 'Baixa':
        return 'sell' 
    else:
        return 0

def teste(x):
    if x['RSI'] < 25 and x['Tendencia'] == 'Alta':
        return 'call'
    elif x['RSI'] > 75 and x['Tendencia'] == 'Baixa':
        return 'sell' 
    else:
        return 0

def resultado_var(x):
    if x['pos_mfi_rsi_stoch_tend'] == 'call':
        resultado = x['close_distancia']/x['Close']
        return resultado
    elif x['pos_mfi_rsi_stoch_tend'] == 'sell':
        resultado = x['Close']/ x['close_distancia']
        return resultado
    else:
        return 0

def resultado_qtd(x):
    if x['pos_mfi_rsi_stoch_tend'] == 'call':
        resultado = x['close_distancia'] - x['Close']
        return resultado
    elif x['pos_mfi_rsi_stoch_tend'] == 'sell':
        resultado = x['Close'] - x['close_distancia']
        return resultado
    else:
        return 0

tempo_distancia = -10

df['MA_200'] = ta.SMA(df['Close'],timeperiod=200)
df['RSI'] = ta.RSI(df['Close'],14)
df['MFI'] = ta.MFI(df['High'], df['Low'], df['Close'], df['Vol'], timeperiod=14)
df['slowk'], df['slowd'] = ta.STOCH(df['High'], df['Low'], df['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
df['Tendencia'] = df.apply(tendencia,axis=1)
df['close_distancia'] = df['Close'].shift(tempo_distancia)
df['pos_mfi_rsi_stoch_tend'] = df.apply(teste, axis = 1)

# Adicionar script para deixar apenas o caso de primeira execução

df['Resultado_var'] = df.apply(resultado_var, axis = 1)
df['Resultado_qtd'] = df.apply(resultado_qtd, axis = 1)


In [221]:
qtd_compras = df[(df['Resultado_var'] != 0) & (df['pos_mfi_rsi_stoch_tend'] == 'call')].shape[0]
qtd_compras_lucro = df[(df['Resultado_var'] != 0) & (df['pos_mfi_rsi_stoch_tend'] == 'call') & (df['Resultado_qtd'] > 0)].shape[0]

print(f'Total de compras: {qtd_compras} \nTotal de compras com lucro: {qtd_compras_lucro} \nPorcentagem de acerto:  {100 * qtd_compras_lucro/qtd_compras}%')

Total de compras: 87 
Total de compras com lucro: 49 
Porcentagem de acerto:  56.32183908045977%


In [222]:
qtd_compras = df[(df['Resultado_var'] != 0) & (df['pos_mfi_rsi_stoch_tend'] == 'sell')].shape[0]
qtd_compras_lucro = df[(df['Resultado_var'] != 0) & (df['pos_mfi_rsi_stoch_tend'] == 'sell') & (df['Resultado_qtd'] > 0)].shape[0]

print(f'Total de vendas: {qtd_compras} \nTotal de vendas com lucro: {qtd_compras_lucro} \nPorcentagem de acerto:  {100 * qtd_compras_lucro/qtd_compras}%')

Total de vendas: 105 
Total de vendas com lucro: 51 
Porcentagem de acerto:  48.57142857142857%


In [223]:
df[df['pos_mfi_rsi_stoch_tend'] == 'call']

,Data,Open,High,Low,Close,Vol,MA_200,RSI,MFI,slowk,slowd,Tendencia,pos_mfi_rsi_stoch_tend,close_distancia,Resultado_var,Resultado_qtd
1674,2020-03-20 08:45,1.07321,1.07382,1.07144,1.07188,1700,1.071406,23.732428,28.028240,9.113929,10.610886,Alta,call,1.07455,1.002491,0.00267
9498,2020-04-28 12:55,1.08476,1.08477,1.08395,1.08434,1429,1.084115,24.270501,13.136553,8.903213,9.012959,Alta,call,1.08563,1.001190,0.00129
20865,2020-06-23 00:10,1.12605,1.12616,1.12500,1.12502,1723,1.124724,20.478464,14.265704,3.126950,10.025084,Alta,call,1.12607,1.000933,0.00105
22170,2020-06-29 12:55,1.12618,1.12620,1.12564,1.12578,1268,1.124905,24.840204,8.890180,20.216748,14.303458,Alta,call,1.12504,0.999343,-0.00074
22171,2020-06-29 13:00,1.12579,1.12609,1.12548,1.12569,1159,1.124924,24.151494,4.215446,20.998092,19.336790,Alta,call,1.12483,0.999236,-0.00086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198562,2022-11-06 21:10,0.99106,0.99123,0.99084,0.99095,795,0.985160,19.790018,44.554968,15.106732,19.597729,Alta,call,0.99276,1.001827,0.00181
198563,2022-11-06 21:15,0.99098,0.99143,0.99080,0.99134,421,0.985226,24.822427,43.164184,17.569787,16.505504,Alta,call,0.99246,1.001130,0.00112
198564,2022-11-06 21:20,0.99135,0.99160,0.99109,0.99110,446,0.985291,23.831525,44.242651,35.275893,22.650804,Alta,call,0.99410,1.003027,0.00300
198565,2022-11-06 21:25,0.99111,0.99149,0.99101,0.99117,79,0.985355,24.774764,40.808453,45.547513,32.797731,Alta,call,0.99390,1.002754,0.00273


,Data,Open,High,Low,Close,Vol,MA_200,RSI,MFI,slowk,slowd,Tendencia,pos_mfi_rsi_stoch_tend,close_distancia,Resultado_var,Resultado_qtd
9495,2020-04-28 12:40,1.08608,1.08609,1.08540,1.08559,1250,1.084082,32.114256,19.203744,18.094130,33.202962,Alta,0,1.08546,0.00000,0.00000
9496,2020-04-28 12:45,1.08558,1.08561,1.08505,1.08524,1172,1.084095,29.638313,13.468643,10.382742,21.689895,Alta,0,1.08555,0.00000,0.00000
9497,2020-04-28 12:50,1.08523,1.08524,1.08473,1.08476,1251,1.084106,26.608468,13.463768,7.752922,12.076598,Alta,0,1.08542,0.00000,0.00000
9498,2020-04-28 12:55,1.08476,1.08477,1.08395,1.08434,1429,1.084115,24.270501,13.136553,8.903213,9.012959,Alta,call,1.08563,1.00119,0.00129
